In [1]:
#import TM_base_calc
from TM_base_calc import *

In [22]:
consideration_list = ['Aerobraked Ammonia Asteroid', 'Artificial Lake', 
                     'Business Contacts', 'Commericial District', 'Ecological Zone', 
                     'Eos Chasma National Park', 'Ganymede Colony', 
                     'Imported Hydrogen', 'Imported Nitrogen',
                     'Indentured Workers', 'Industrial Center', 'Invention Contest', 
                     'Large Convoy', 'Local Heat Trapping', 'Mangrove', 
                     'Mining Area', 'Mining Rights', 'Mohole Area', 
                     'Natural Preserve',
                     'Noctis City', 'Nuclear Zone', 'Phobos Space Haven',  
                     'Protected Valley', 'Research Outpost', 'Restricted Area',  
                     'Urbanized Area']
db_list = []
for i in range(1, len(base_cards)):
    if base_cards.Title.iloc[i] in consideration_list:
        db_list.append(base_cards.iloc[i])
db = pd.DataFrame(db_list)
display(db)

,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent
5,Aerobraked Ammonia Asteroid,base,event,26,NaN,"space, event",NaN,NaN,"2 microbe on other card, 3 HEAT, 1 PLANT",NaN,NaN,NaN,NaN
13,Artificial Lake,base,automated,15,NaN,building,-6°C,1,1 ocean on site not reserved for ocean,NaN,NaN,NaN,NaN
27,Business Contacts,corporate,event,7,NaN,"earth, event",NaN,NaN,"4 draw, then discard 2 of them",NaN,NaN,NaN,NaN
51,Ecological Zone,base,active,12,NaN,"animal, plant",player’s 1 greenery tile,1 per 2 animal on card,special tile adjacent to any greenery,"when playing an animal or plant tag, 1 animal to card",NaN,NaN,NaN
55,Eos Chasma National Park,base,automated,16,NaN,"plant, building",-12°C,1,"1 animal on other card, 3 plant, 2 CREDIT",NaN,NaN,NaN,NaN
65,Ganymede Colony,base,automated,20,NaN,"jovian, space, city",NaN,1 per player jovian tag,1 city on reserved site,NaN,NaN,NaN,NaN
87,Imported Hydrogen,base,event,16,NaN,"earth, space, event",NaN,NaN,"3 plant or 3 microbe on other card or 2 animal on other card, 1 ocean",NaN,NaN,NaN,NaN
88,Imported Nitrogen,base,event,23,NaN,"earth, space, event",NaN,NaN,"1 TR, 4 plant, 3 microbe on other card, 2 animal on other card",NaN,NaN,NaN,NaN
89,Indentured Workers,corporate,event,0,NaN,event,NaN,-1,the next card played by player costs 8 credit fewer,NaN,NaN,NaN,NaN
90,Industrial Center,corporate,active,4,NaN,building,NaN,NaN,special tile adjacent to any city,NaN,7 credit,1 STEEL,NaN


In [37]:
# pers

def pers(i_benefit_list, generation, player_count):
    income, vp, TR = initialize(generation, player_count)
    for i in range(len(i_benefit_list)):
        if i_benefit_list[i] == 'per':
            benefit_a = vars_scan(i_benefit_list)
                
            if any('tag' in entry for entry in i_benefit_list):
                for x in range(len(i_benefit_list)):
                    if i_benefit_list[x] == 'player' or i_benefit_list[x] == 'opponent':
                        for tag in tags_dict:
                            if tag == i_benefit_list[x+1]:
                                benefit_b = int(i_benefit_list[x-1]) * \
                                               (tags_dict[tag] * (10 + 4 * (generation - 1)))
                                
            elif any('city' in entry for entry in i_benefit_list):
                for x in range(len(i_benefit_list)):
                    if i_benefit_list[x] == 'city':
                        benefit_b = int(i_benefit_list[x-1]) * \
                                        cities_current(generation, player_count)
                        
            elif any('event' in entry for entry in i_benefit_list):
                for x in range(len(i_benefit_list)):
                    if i_benefit_list[x] == 'event':
                        benefit_b = int(i_benefit_list[x-1]) * \
                        (chance_event * ((10 + 4 * (generation - 1)) * 2))
                        
            points_per = benefit_a * benefit_b
            return points_per

In [35]:
def ors(or_benefit, generation, player_count):
    income, vp, TR = initialize(generation, player_count)
    or_list = or_benefit.split(' or ')
    or_value, or_val = [0, 0]
    for benefit in or_list:
        benefit = benefit.split(' ')
        or_val = vars_scan(benefit)
        if or_val > or_value: or_value = or_val
    return or_value

In [4]:
# determine current quarter of the game
def game_quarter(generation, player_count):
    quarters_list = []
    for gen in gen_quarters:
        if gen[0] == str(player_count):
            quarters_list.append(gen_quarters[gen])
    absolute_difference_function = lambda list_value : abs(list_value - generation)
    for i in range(4):
        if quarters_list[i] == min(quarters_list, key=absolute_difference_function):
            quarter = i+1
    return quarter

#determine credits per vp based on quarter
def current_cpp(quarter, player_count):    
    for cpp in gen_credits_per_vp:
        if cpp[0] == str(player_count) and cpp[3] == str(quarter):
            vp = gen_credits_per_vp[cpp]
    return vp

#run above functions and determine credits per TR
#def initialize(generation, player_count):
#    income = remgen(generation, player_count)
#    quarter = game_quarter(generation, player_count)
#    vp = current_cpp(quarter, player_count)
#    TR = vp + income
#    return income, quarter, vp, TR

In [39]:
# Discarded snippets:

# from main page---
# main value calculation function after line 22 (replaces line 23):
                #if i % 2 != 0:    # pulls variable from vars_dict
                #    for var in vars_dict_static:
                #        if var == add_cost_list[i]:
                #            value -= int(add_cost_list[i-1]) * vars_dict_static[var]
                #    for var in vars_dict_variable:
                #        if var == add_cost_list[i]:
                #            value -= int(add_cost_list[i-1]) * vars_dict_variable[var](income)   
                
# main value calculation function after line 47 (replaces line 48):
                    #for var in vars_dict_static:
                    #    if var == i_benefit_list_inner[i][1]:
                    #        value += int(i_benefit_list_inner[i][0]) * vars_dict_static[var]
                    #for var in vars_dict_variable:
                    #    if var == i_benefit_list_inner[i][1]:
                    #        value += int(i_benefit_list_inner[i][0]) * vars_dict_variable[var](income)
                    #for var in vars_dict_resource:
                    #    if var == i_benefit_list_inner[i][1]:
                    #        value += int(i_benefit_list_inner[i][0]) * vars_dict_resource[var]\
                    #        (generation, player_count)
                    #if i_benefit_list_inner[i][1] == 'TR':
                    #    value += int(i_benefit_list_inner[i][0]) * TR
                    
# from this page---
#ors after line 6 (replaces line 7):
        #for var in vars_dict_static:
        #    if var == benefit[1]:
        #        or_val = int(benefit[0]) * vars_dict_static[var]
        #for var in vars_dict_variable:
        #    if var == benefit[1]:
        #        or_val = int(benefit[0]) * vars_dict_variable[var](income)
        #for var in vars_dict_resource:
        #    if var == benefit[1]:
        #        or_val = int(benefit[0]) * vars_dict_resource[var](generation, player_count)
        
#pers after line 6 (replaces line 7):
            #for var in vars_dict_static:
            #    if var == i_benefit_list[i-1]:
            #        benefit_a = int(i_benefit_list[i-2]) * vars_dict_static[var]
            #for var in vars_dict_variable:
            #    if var == i_benefit_list[i-1]:
            #        benefit_a = int(i_benefit_list[i-2]) * vars_dict_variable[var](income)
            #        
            #if i_benefit_list[i-1] == 'TR':
            #    benefit_a = int(i_benefit_list[i-2]) * TR